In [15]:
import pandas as pd
from pyspark.sql.functions import *
#from mpl_toolkits.basemap import Basemap
#import matplotlib.pyplot as plt
import numpy as np

In [2]:
path="s3://finalgeodata/"

In [3]:
d = sc.textFile(path+"devicestatus.txt").map(lambda x: x.replace('|',',').replace('/',',').split(',')).toDF()
data=d.toPandas()

In [13]:
#data

In [5]:
df=d.select("_13","_14","_1","_2","_3").withColumnRenamed("_13", "latitude").withColumnRenamed("_14", "longitude").withColumnRenamed("_1", "date").withColumnRenamed("_2", "model").withColumnRenamed("_3", "deviceID")

In [6]:
#df.show()

In [7]:
df =df.filter((df.latitude!=0) & (df.longitude!=0))

In [9]:
#df.count()

In [10]:
df=df.withColumn("manufacturer",split(df.model," ").getItem(0)).withColumn("model",split(df.model," ").getItem(1))

In [11]:
df.show()

+-------------+--------------+-------------------+-------+--------------------+------------+
|     latitude|     longitude|               date|  model|            deviceID|manufacturer|
+-------------+--------------+-------------------+-------+--------------------+------------+
|33.6894754264|-117.543308253|2014-03-15:10:10:20|   F41L|8cc3b47e-bd01-448...|    Sorrento|
|37.4321088904|-121.485029632|2014-03-15:10:10:20|    1.0|ef8c7564-0a1a-465...|      MeeToo|
|39.4378908349|-120.938978486|2014-03-15:10:10:20|    1.0|23eba027-b95a-472...|      MeeToo|
|39.3635186767|-119.400334708|2014-03-15:10:10:20|   F41L|707daba1-5640-4d6...|    Sorrento|
|33.1913581092|-116.448242643|2014-03-15:10:10:20|Novelty|db66fe81-aa55-43b...|       Ronin|
|33.8343543748|-117.330000857|2014-03-15:10:10:20|   F41L|ffa18088-69a0-433...|    Sorrento|
|37.3803954321|-121.840756755|2014-03-15:10:10:20|   F33L|66d678e6-9c87-48d...|    Sorrento|
|34.1841062345|  -117.9435329|2014-03-15:10:10:20|    4.1|673f7e4b-d52

In [15]:
df.write.csv("s3a://finalgeodata/outputstep1")